## Finding differentialy expressed miRNA's between LUAD stages I, II, III, IV vs normal using Logit regression
### Utilize Group Lasso with MISIM miRNA similarity data

In [166]:
# Necessary imports
import os
import pandas
import numpy as np

from definitions import ROOT_DIR

## Load normal and cancer miRNA expression data

In [167]:
from definitions import ROOT_DIR

# Load files into pandas data frames
mirna_tumor_df = pandas.read_csv(os.path.join(ROOT_DIR, "data/processed/miRNA/tumor_miRNA.csv"))
mirna_normal_df = pandas.read_csv(os.path.join(ROOT_DIR, "data/processed/miRNA/normal_miRNA.csv"))
clinical_df = pandas.read_csv(os.path.join(ROOT_DIR, "data/processed/clinical/clinical.csv"))
validated_miRNA_csv = pandas.read_csv(os.path.join(ROOT_DIR, 'data/external/validated_luad_miRNAs_miRCancer.csv'))

# Print data frame shapes
print "mirna_tumor_df.shape", mirna_tumor_df.shape, ', nulls:', mirna_tumor_df.isnull().sum().sum()
print "mirna_normal_df.shape", mirna_normal_df.shape, ', nulls:', mirna_normal_df.isnull().sum().sum()
print 'validated_miRNAs.shape', validated_miRNA_csv.shape

# Merge normal and tumor miRNA expression profiles with clinical cancer stage data
mirna_normal = pandas.merge(clinical_df[['patient_barcode', 'pathologic_stage']], mirna_normal_df, on='patient_barcode')
mirna_normal['pathologic_stage'] = 'normal'
mirna_tumor = pandas.merge(clinical_df[['patient_barcode', 'pathologic_stage']], mirna_tumor_df, on='patient_barcode')

# Map stage IA to stage I, IB to I, etc. ...
pathologic_stage_map = {'Stage IA': 'Stage I', 'Stage IB': 'Stage I', 
                        'Stage IIA': 'Stage II', 'Stage IIB': 'Stage II', 
                        'Stage IIIA': 'Stage III', 'Stage IIIB': 'Stage III'}
mirna_tumor.replace({'pathologic_stage': pathologic_stage_map}, inplace=True)

# Store list of all miRNA's in miRNA expression data
mirna_list = list(mirna_tumor.columns)[2:]

# Print quick overview of data
print mirna_normal['pathologic_stage'].value_counts().sort_index(axis=0)
print mirna_tumor['pathologic_stage'].value_counts().sort_index(axis=0)

mirna_tumor_df.shape (513, 1882) , nulls: 0
mirna_normal_df.shape (46, 1882) , nulls: 0
validated_miRNAs.shape (34, 2)
normal    46
Name: pathologic_stage, dtype: int64
Stage I      277
Stage II     121
Stage III     84
Stage IV      24
Name: pathologic_stage, dtype: int64


## Load normal & cancer Gene Expression data

In [168]:
gene_exp_tumor_df = pandas.read_table(os.path.join(ROOT_DIR, 'data/processed/gene_expression/tumor/READ__illuminahiseq_rnaseqv2__GeneExp.txt'), 
                                      header=0, delimiter='\t')
gene_exp_normal_df = pandas.read_table(os.path.join(ROOT_DIR, 'data/processed/gene_expression/normal/READ__illuminahiseq_rnaseqv2__GeneExp.txt'), 
                                      header=0, delimiter='\t')

gene_exp_tumor_df.rename(columns=lambda x: x[:12], inplace=True)
gene_exp_normal_df.rename(columns=lambda x: x[:12], inplace=True)

print gene_exp_tumor_df.shape
print gene_exp_normal_df.shape

(20530, 517)
(20530, 61)


## Reshape gene expression data frames to have shape (patients x gene expression) 

In [169]:
print len(list(gene_exp_tumor_df.columns))-2
print len(list(gene_exp_normal_df.columns))-2

# Remove entries with unknown Gene Symbol
gene_exp_tumor_df = gene_exp_tumor_df[gene_exp_tumor_df.GeneSymbol != '?']
gene_exp_normal_df = gene_exp_normal_df[gene_exp_normal_df.GeneSymbol != '?']

# Get list of all gene_symbols
gene_symbols = list(gene_exp_tumor_df['GeneSymbol'])
# Get list of tumor and normal patient_barcode
gene_exp_tumor_patient_barcodes = list(gene_exp_tumor_df.columns)[2:]
gene_exp_normal_patient_barcodes = list(gene_exp_normal_df.columns)[2:]

# Drop EntrezID column
gene_exp_tumor = gene_exp_tumor_df.drop(['EntrezID', 'GeneSymbol'], axis=1)
gene_exp_normal = gene_exp_normal_df.drop(['EntrezID', 'GeneSymbol'], axis=1)

# Reshaping data frame to have columns for GeneSymbols, and rows of patients
gene_exp_tumor = gene_exp_tumor.T
gene_exp_normal = gene_exp_normal.T
gene_exp_tumor.columns = gene_symbols
gene_exp_normal.columns = gene_symbols

# Add column for patients barcode
gene_exp_tumor['patient_barcode'] = gene_exp_tumor.index
gene_exp_normal['patient_barcode'] = gene_exp_normal.index

print "gene_symbols", len(gene_symbols)
print "gene_exp_tumor_patients", len(gene_exp_tumor_patient_barcodes)
print "gene_exp_normal_patients", len(gene_exp_normal_patient_barcodes)

print gene_exp_tumor.shape
print gene_exp_normal.shape

515
59
gene_symbols 20502
gene_exp_tumor_patients 515
gene_exp_normal_patients 59
(515, 20503)
(59, 20503)


## Match Gene Expression data to miRNA data

In [170]:
# Merge normal and tumor miRNA expression profiles with clinical cancer stage data
merged_normal_patients = pandas.merge(gene_exp_normal[['patient_barcode']], mirna_normal, on='patient_barcode')[['patient_barcode', 'pathologic_stage']]
merged_tumor_patients = pandas.merge(gene_exp_tumor[['patient_barcode']], mirna_tumor, on='patient_barcode')[['patient_barcode', 'pathologic_stage']]

# Print quick overview of data
print merged_normal_patients['pathologic_stage'].value_counts().sort_index(axis=0)
print merged_tumor_patients['pathologic_stage'].value_counts().sort_index(axis=0)

normal    20
Name: pathologic_stage, dtype: int64
Stage I      275
Stage II     120
Stage III     84
Stage IV      24
Name: pathologic_stage, dtype: int64


## Build miRNA-target relationship network to identify 
## Xu et al. (xu2011prioritizing)

In [171]:
from src.models.miRNA_target_network import miRNATargetNetwork
import networkx as nx

network = miRNATargetNetwork(threshold=0.6)
network.train(miRNAs_A=mirna_tumor, targets_A=gene_exp_tumor, miRNAs_B=mirna_normal, targets_B=gene_exp_normal)
print nx.bipartite.sets(network.B)

TypeError: Could not convert TCGA-53-7813TCGA-49-6745TCGA-44-A47GTCGA-55-7910TCGA-97-7547TCGA-L4-A4E6TCGA-67-4679TCGA-55-8094TCGA-91-6831TCGA-55-7724TCGA-78-7147TCGA-44-6778TCGA-55-8206TCGA-78-7540TCGA-44-5645TCGA-67-6215TCGA-99-AA5RTCGA-73-A9RSTCGA-MP-A4T4TCGA-97-7552TCGA-78-7153TCGA-93-A4JOTCGA-44-6146TCGA-93-A4JPTCGA-93-7348TCGA-MP-A4TCTCGA-80-5607TCGA-J2-8194TCGA-78-7166TCGA-73-7499TCGA-49-6761TCGA-97-A4M5TCGA-62-A46OTCGA-78-7535TCGA-67-3774TCGA-MP-A4TJTCGA-MP-A4TITCGA-55-8299TCGA-62-8397TCGA-55-1594TCGA-86-8359TCGA-55-6972TCGA-44-2668TCGA-55-7574TCGA-05-4420TCGA-64-1679TCGA-97-8176TCGA-62-A46RTCGA-91-6828TCGA-L4-A4E5TCGA-05-4410TCGA-50-5932TCGA-78-8660TCGA-44-6774TCGA-55-8097TCGA-05-4390TCGA-75-6207TCGA-55-7913TCGA-55-8511TCGA-44-8117TCGA-38-4626TCGA-75-7027TCGA-97-7554TCGA-97-8552TCGA-91-6836TCGA-95-7562TCGA-44-2662TCGA-NJ-A55ATCGA-44-6775TCGA-55-6969TCGA-55-A492TCGA-55-A491TCGA-73-4676TCGA-L9-A8F4TCGA-91-A4BDTCGA-L9-A444TCGA-78-7163TCGA-69-7763TCGA-NJ-A4YGTCGA-05-4249TCGA-55-A4D

In [173]:
nx.bipartite.sets(network.B)

(set(),
 {'MTVR2',
  'ATRX',
  'LOC441204',
  'TCOF1',
  'NSRP1',
  'LOC441208',
  'SPPL3',
  'OPA3',
  'OPA1',
  'hsa-mir-4707',
  'hsa-mir-4706',
  'hsa-mir-4705',
  'hsa-mir-4704',
  'hsa-mir-4703',
  'hsa-mir-4701',
  'hsa-mir-4700',
  'ITGA1',
  'ITGA2',
  'ITGA3',
  'ITGA4',
  'ITGA5',
  'hsa-mir-4709',
  'hsa-mir-4708',
  'TRHR',
  'LOC100132288',
  'UFSP1',
  'UFSP2',
  'FAM212B',
  'FAM212A',
  'CHST9',
  'CHST8',
  'CHST1',
  'CHST3',
  'CHST2',
  'CHST5',
  'CHST4',
  'CHST7',
  'CHST6',
  'ITGAX',
  'BCL2A1',
  'ITGAV',
  'PREP',
  'ITGAL',
  'ITGAM',
  'ITGAD',
  'SPR',
  'IGF2R',
  'SLC36A4',
  'SLC36A3',
  'CKS1B',
  'SLC36A1',
  'C14orf119',
  'ART1',
  'ART3',
  'ART5',
  'ART4',
  'hsa-mir-3168',
  'hsa-mir-3169',
  'hsa-mir-3166',
  'hsa-mir-3167',
  'hsa-mir-3164',
  'hsa-mir-3165',
  'hsa-mir-3162',
  'hsa-mir-3163',
  'hsa-mir-3161',
  'MGC57346',
  'CRTAM',
  'CADPS2',
  'RLF',
  'CRTAP',
  'MDH1',
  'HPYR1',
  'CPEB4',
  'CPEB1',
  'CPEB3',
  'SBSPON',
  'SRBD1'

## Loading miRecords experimentally validated miRNA-targets interactions

In [157]:
# Load data frame from file
miRecords_df = pandas.read_table(os.path.join(ROOT_DIR, 'data/external/miRecords_version4.tsv'), delimiter='\t')

# Select only homo sapiens miRNA-target pairs
miRecords_df = miRecords_df[(miRecords_df["miRNA_species"] == "Homo sapiens") &
                            (miRecords_df["Target gene_species_scientific"] == "Homo sapiens")]
miRecords_df = miRecords_df[["miRNA_mature_ID", "Target gene_name"]]

# Standardize miRNA and gene symbols
miRecords_df['miRNA_mature_ID'] = miRecords_df['miRNA_mature_ID'].str.lower()
miRecords_df['miRNA_mature_ID'] = miRecords_df['miRNA_mature_ID'].str.replace('*', '')
miRecords_df['Target gene_name'] = miRecords_df['Target gene_name'].str.upper()

# Filter miRNA-target pairs to only miRNA's included in miRNA expression data, same for gene targets 
miRecords_df = miRecords_df[miRecords_df["miRNA_mature_ID"].isin(mirna_list) &
                            miRecords_df["Target gene_name"].isin(gene_symbols)]

# miRecords_df